In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import scipy.ndimage.filters as filters

In [2]:
sat_14 = xr.open_dataset('/home/travis/python_code/forecasting/current_data/sat_14.nc')
sat_15 = xr.open_dataset('/home/travis/python_code/forecasting/current_data/sat_15.nc')
wind_15 = xr.open_dataset('/home/travis/python_code/forecasting/current_data/wind_15_crop.nc')
sensor_data = pd.read_hdf('/home/travis/python_code/forecasting/current_data/sensor_data.h5')
sensor_loc = pd.read_hdf('/home/travis/python_code/forecasting/current_data/sensor_loc.h5')

In [3]:
clear_sky_good = xr.DataArray(
    data=(sat_15.GHI.values/sat_14.GHI.values).clip(max=1),
    coords=sat_15.coords)
clear_sky_good = clear_sky_good.rename({'x': 'y_', 'y': 'x_'})
clear_sky_good = clear_sky_good.rename({'y_': 'y', 'x_': 'x'})
sat_15['clear_sky_good'] = clear_sky_good
sat_15 = sat_15.rename({'x': 'west_east', 'y': 'south_north'})

In [4]:
sensor_CI = sensor_data[['clearsky_index', 'id']]
sensor_CI = sensor_CI.reset_index().pivot(
    index='time', columns='id', values='clearsky_index')
sensor_CI = sensor_CI.interpolate(method='time')
sensor_CI = sensor_CI.resample('5min').mean().dropna()

In [6]:
U = filters.uniform_filter(wind_15.U, (0, 300, 300), mode='mirror')
V = filters.uniform_filter(wind_15.V, (0, 300, 300), mode='mirror')

wind_15_smooth = wind_15.copy()

wind_15_smooth['U'] = (wind_15.U.dims, U)
wind_15_smooth['V'] = (wind_15.V.dims, V)

In [7]:
wind_ave = wind_15.mean(dim='south_north').mean(dim='west_east').mean(dim='south_north_stag').mean(dim='west_east_stag')

U_ave = wind_ave.U.values
V_ave = wind_ave.V.values

wind_15_ave = wind_15.copy()

U_ave = np.ones_like(wind_15.U.values)*wind_ave.U.values[:, None, None]
V_ave = np.ones_like(wind_15.V.values)*wind_ave.V.values[:, None, None]

wind_15_ave['U'] = (wind_15.U.dims, U_ave)
wind_15_ave['V'] = (wind_15.V.dims, V_ave)

In [8]:
sensor_CI.to_hdf('sensor_CI.h5', 'sensor_CI')

In [9]:
sat_15.to_netcdf('sat_15_good.nc')
wind_15_smooth.to_netcdf('wind_15_smooth.nc')
wind_15_ave.to_netcdf('wind_15_ave.nc')